In [1]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./auth.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
input_['Auth'].tolist()

总数量：166


['mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 'mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw',
 

In [3]:
crawler_status = 'error'

print(crawler_status)

error


In [4]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [5]:
a = 0

resp = requests.get('https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:' + input_.loc[a, 'Part_Number'] + ')&size=5000&useCache=true&auth=' + input_.loc[a, 'Auth'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'[]'

In [6]:
import json

In [7]:
json_data = json.loads(resp)

json_data

[]

In [8]:
dict_vehicle = {}
for data in json_data:
    make = data['fitment']['Vcdb']['MakeName'].strip()
    model = data['fitment']['Vcdb']['ModelName'].strip()
    if len(data['fitment']['Vcdb']['Engines']) == 0:
        engine = ''
    else:
        engine = '' if data['fitment']['Vcdb']['Engines'][0]['Liter'] == 0 else str(data['fitment']['Vcdb']['Engines'][0]['Liter']) + 'L'
        engine += '' if data['fitment']['Vcdb']['Engines'][0]['BlockType'].strip() == '-' else ' ' + data['fitment']['Vcdb']['Engines'][0]['BlockType'].strip() + str(data['fitment']['Vcdb']['Engines'][0]['Cylinders'])
    year = int(data['fitment']['Vcdb']['Year'])
    
    if make not in dict_vehicle:
        dict_vehicle[make] = {model: {engine: [year]}}
    else:
        if model not in dict_vehicle[make]:
            dict_vehicle[make][model] = {engine: [year]}
        else:
            if engine not in dict_vehicle[make][model]:
                dict_vehicle[make][model][engine] = [year]
            else:
                dict_vehicle[make][model][engine] += [year]
                
    dict_vehicle[make][model][engine] = sorted(list(set(dict_vehicle[make][model][engine])))
    
dict_vehicle

{}

In [9]:
list_vehicle = []
for make in dict_vehicle:
    for model in dict_vehicle[make]:
        list_year = []
        for engine in dict_vehicle[make][model]:
            list_year += dict_vehicle[make][model][engine]
        list_year = sorted(list(set(list_year)))
        
        year_min = min(list_year)
        year_max = max(list_year)
        if year_min == year_max:
            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
        else:
            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

vehicle_1 = '\n'.join(sorted(list_vehicle))

print(vehicle_1)

In [10]:
list_vehicle = []
for make in dict_vehicle:
    for model in dict_vehicle[make]:
        for engine in dict_vehicle[make][model]:
            year_min = min(dict_vehicle[make][model][engine])
            year_max = max(dict_vehicle[make][model][engine])
            if year_min == year_max:
                if engine == '':
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                else:
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
            else:
                if engine == '':
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                else:
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

vehicle_2 = '\n'.join(sorted(list_vehicle))

print(vehicle_2)

In [11]:
df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                         'Vehicle_1': vehicle_1,
                         'Vehicle_2': vehicle_2}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

1

In [12]:
crawler_status = 'ok'

print(crawler_status)

ok


In [13]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                             'Auth': input_.loc[a, 'Auth']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

0

In [14]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,Part_Number,Vehicle_1,Vehicle_2
0,43-1429,,


In [15]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
